# This notebook contains codes to combine the three retina datasets: DRIVE, STARE and CHASE

1. Resize all images to the same size.  
2. Convert all images to jpg. 
3. The label / mask images are converted to 1 channel (grayscale) as required by the train.py   

In [1]:
import os

def get_all_images_in_dir(folder = '../data/fundus/images_public/', target_file = None, excludes = ['_FRCNN','_SSD','_YOLO5']):

	'''
	excludes : will exclude images whose names contains these strings.
	'''

	FILES=[]	
	content = ""

	for root, dirs, files in os.walk(folder):
		for f in files:
			if( os.path.isfile(os.path.join(root, f)) and (f.endswith('.jpg') or f.endswith('.png') or f.endswith('.ppm') or f.endswith('.gif') or f.endswith('.tif')) ):
				fp =  os.path.join(root, f).replace("\\", "/") 
				
				exclude = False
				if excludes is not None and len(excludes) > 0:
					for s in excludes:
						if s in f:
							exclude = True
							break
				
				if not exclude:
					FILES.append(fp)
					content = content + fp + ",-1,-1,-1,-1,UNKNOWN" + "\n" # filename,x1,y1,x2,y2,class_name

	if target_file:
		file = open(target_file, 'w')
		file.write(content)
		file.close()

	return FILES

In [2]:
FILES = get_all_images_in_dir(folder = '.')
len(FILES)

176

In [4]:
import cv2
import imageio


H = 584
W = 565
size = (W, H)

for path in FILES:
    
    if '.gif' in path: # special treatment for gif. gif is multi-sequence.
        gif = imageio.mimread(path)
        nums = len(gif)
        # convert form RGB to BGR 
        imgs = [cv2.cvtColor(img, cv2.COLOR_RGB2BGR) for img in gif]
        im = imgs[0]
    elif '.tif' in path:
        im = cv2.imread(path, cv2.IMREAD_ANYDEPTH | cv2.IMREAD_UNCHANGED)
        #im = rasterio.open(path)
    else:
        im = cv2.imread(path)
    
    if im is None:
        print(path)
        
    if 'label' in path: # the masks need to be 1 channel
        im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY) 
        im.dtype='uint8' # actually should be binary
    
    # OH, OW, _ = image.shape # H, W, C
    # keep_aspect_size = (round(H*OW/OH), H)
    im = cv2.resize(im, size)
    cv2.imwrite(path +'.jpg', im) 

ValueError: Could not find a format to read the specified file in multi-image mode

In [39]:
import os

for path in FILES:
    os.remove(path)

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'fundus/ALL/test_data/01_test.tif'